# Flow ML Course

## TP 3 : Vertex AI  SDK Python

L'objectif de ce notebook est d'apprendre à utiliser les experiments de Flow ML.

Les Experiments permettent de logger les méta paramètres des architectures des modèles et leurs métriques. On peut regrouper les différents entrainements sous une experiment pour les comparer.


En savoir plus [Experiments]( https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments).

En savoir plus [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction).

In [4]:
PROJECT_ID = "projet-ia-448520" # "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

BUCKET_URI = "gs://cours1bucket" # f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

SERVICE_ACCOUNT = "837843744498-compute@developer.gserviceaccount.com" # "[your-service-account]"  # @param {type:"string"}

In [ ]:
import os
import uuid

import google.cloud.aiplatform as aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

#### Create experiment for tracking training related metadata

First, you create an experiment using the `init()` method and then initialize a run within the experiment using `start_run()`.

- `aiplatform.init()` - Create an experiment instance
- `aiplatform.start_run()` - Track a specific run within the experiment.

### Create artifact lineage in experiment runs

Créeons un lineage pour une experiment fictive

In [ ]:
# Create experiment
your_name = "hugo"
EXPERIMENT_NAME = f"supaero-vertex-course-1-3-{your_name}"
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

## Create Artifcacts Create a dataset and model artifacts

Créons des artifacts synthétiques pour simuler un lineage :

- `dataset_artifact`: Dataset d'input de l'Experiment Run 
- `model_artifact`: Model d'output de l'Experiment Run 

In [27]:
DATASET_URI = "gs://example/dataset.csv"
MODEL_URI = "gs://example/saved_model.pb"

dataset_artifact = aiplatform.Artifact.create(
    schema_title="system.Dataset", display_name="example_dataset", uri=DATASET_URI
)

model_artifact = aiplatform.Artifact.create(
    schema_title="system.Model", display_name="example_modl", uri=MODEL_URI
)

## Artifact lineage

Procédure :
- initialiser l'éxécution : `start_execution()`
- Attacher à la run l'input `assign_input_artifacts()` 
- Attacher à la run l'output `assign_output_artifacts()` 


In [28]:
with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    execution.assign_input_artifacts([dataset_artifact])

    # Log the params you expect
    hyperparams = {}
    hyperparams["batch_size"] = 64
    hyperparams["learning_rate"] = .01
    aiplatform.log_params(hyperparams)
    
    # Log the metrics you expect
    metrics = {}
    metrics["test_acc"] = 96.7
    metrics["train_acc"] = 97.1
    aiplatform.log_metrics(metrics)
    

    execution.assign_output_artifacts([model_artifact])

    aiplatform.log_metrics(
        {"lineage": execution.get_output_artifacts()[0].lineage_console_uri}
    )

## Extraire les résultats d'un Experiment

Pour terminer une expirement : utiliser `end_run()` 

Ensuite, vous pouvez récupérer la plupart des informations de l'experiment via : `get_experiment_df()` 


In [29]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

                                                                 0
experiment_name       example-81009a18-e9fa-11ef-8f5e-42010a800004
run_name                                                     run-1
run_type                                      system.ExperimentRun
state                                                     COMPLETE
param.units                                                  256.0
metric.acc                                                    96.8
metric.lineage   https://console.cloud.google.com/vertex-ai/loc...


## Visualiser l'Artifact Lineage

Ouvrez le lien suivant pour suivre l'artifcat lineage

In [30]:
print(
    "Open the following link:", execution.get_output_artifacts()[0].lineage_console_uri
)

Open the following link: https://console.cloud.google.com/vertex-ai/locations/us-central1/metadata-stores/default/artifacts/e1ceca8d-d8f3-4abc-ab72-d98beda9b32d?project=projet-ia-448520


#### Supprimer l'Artifact lineage


In [ ]:
try:
    dataset_artifact.delete()
except Exception as e:
    print(e)
try:
    model_artifact.delete()
except Exception as e:
    print(e)

#### Supprimer l'Experiment


In [ ]:
try:
    exp.delete()
except Exception as e:
    print(e)